In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch

In [9]:
df = pd.read_csv("db.csv", header = None).sample(frac=1)
df

,0,1,2,3,4,5,6,7,8,9,...,279,280,281,282,283,284,285,286,287,288
4330,1,115.628353,113.238390,116.449407,115.865305,114.682029,111.761514,111.413909,113.489294,114.904357,...,55.250927,60.158549,57.874076,55.813396,52.700084,52.341777,58.172134,65.249865,76.823719,80.415766
21444,3,78.585713,77.767123,80.188438,77.641645,74.291874,73.117396,74.246029,74.349836,76.238624,...,257.415513,257.746635,256.833318,256.335633,255.969415,250.163701,246.107198,243.057233,245.958179,249.631091
10971,2,215.678180,213.776407,214.342145,209.538481,208.942580,207.037241,204.654999,204.468849,205.165321,...,212.028157,212.822197,216.490818,221.261317,220.003529,214.275346,213.336999,217.140803,227.996443,238.964367
4850,1,99.067611,97.743596,100.373498,99.606292,98.045094,97.161610,97.126649,97.894199,100.580913,...,82.628735,83.785454,81.825665,81.681369,83.619799,89.176209,96.553741,102.400485,106.944249,106.636442
11439,2,102.332735,98.950395,100.461567,96.952608,93.747231,91.412450,91.968154,91.249457,92.672144,...,283.419526,283.721843,281.636070,282.114709,281.476610,283.606558,287.140846,292.612640,304.718121,316.752429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18261,2,129.394233,122.987134,123.837961,126.021344,125.365396,122.382556,124.069548,126.506990,131.450063,...,98.571643,98.548215,97.935691,96.556604,94.627458,91.090445,90.849116,93.128254,96.942249,97.066968
20342,3,161.401651,157.709191,159.297492,160.197528,168.324223,171.475817,175.505143,183.429189,191.829007,...,179.195577,182.448153,188.270346,193.901262,196.145870,196.345651,200.569508,208.895710,221.114061,229.596639
30788,4,284.911645,283.096370,286.193964,284.821585,283.189546,280.087962,276.504785,271.852297,270.380008,...,217.078184,222.916026,226.991793,225.271896,223.478987,212.714892,202.774687,187.494491,172.781046,162.458049
37227,4,148.898851,146.191083,151.789934,151.997154,150.651973,148.935642,146.192135,145.182419,148.031598,...,127.378604,127.419170,127.147583,131.134913,134.305759,133.156142,131.980707,133.947574,133.348522,127.252797
